In [40]:
#Use the train.tsv file at
#https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/data
#-Load the data, show the first five rows, check the data, and provide information on the
#data.

import pandas as pd

df = pd.read_csv('train.tsv', sep = '\t')
df.head(5)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


PhraseID is an index

The data is broken down into fragments of sentences, sorted by SentenceID

Phrase is the sentence fragment

Sentiment is what the sentence is trying to convey, sorted into numbered categories:

0 - negative

1 - somewhat negative

2 - neutral

3 - somewhat positive

4 - positive

In [67]:
#-Show the distribution of review documents.

pd.DataFrame(df.groupby(['Sentiment']).count()['PhraseId'])

,PhraseId
Sentiment,
0,7072
1,27273
2,79582
3,32927
4,9206


In [51]:
#-Generate the document term matrix by using scikitlearn’s CountVectorizer. From
#nltk.tokenizer import RegexpTokenizer. ‘Sentiment’ is the data. Train, test and split.

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer #not sure what this was supposed to be used for? Was concerned that it was
#returning some stuff like 000, but I'm not actually sure if this solves it or if that's even an issue

vect = CountVectorizer()
vects = vect.fit_transform(df['Phrase'].values)

td = pd.DataFrame(vects.todense())
td.columns = vect.get_feature_names_out()
term_document_matrix = td.T
term_document_matrix.columns = ['Phrase' + str(i) for i in range(1, 156061)]

x = np.swapaxes(term_document_matrix, 0, 1)
y = df['Sentiment'].values

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state = 1234)

(156060,)
(156060, 15240)


In [52]:
#-Import the MultinomialNB module and create a MNB classifier object using the
#MultinomialNB() function.

from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB()

In [56]:
#Import scikit.learn metrics to compute the accuracy and fit
#the model before computing the accuracy. What is the outcome?

from sklearn import metrics

y_pred = clf.predict(x_test)

print(metrics.accuracy_score(y_test, y_pred))

0.6068819684736639


In [57]:
#-Use TfidfVectorizer-transformed data and split it into training and test datasets.

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer()
vects = vect.fit_transform(df['Phrase'].values)

td = pd.DataFrame(vects.todense())
td.columns = vect.get_feature_names_out()
term_document_matrix = td.T
term_document_matrix.columns = ['Phrase' + str(i) for i in range(1, 156061)]

x = np.swapaxes(term_document_matrix, 0, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state = 1234)

In [58]:
#-Build the text classification model using TF-IDF. First, import the MultinomialNB
#module and create the MNB classifier object using the MultinomialNB function. Fit the
#model on the training set and perform the prediction. Is the accuracy better?

clf = MultinomialNB()
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print(metrics.accuracy_score(y_test, y_pred))

0.5854158656926823


No, it seems to be slightly worse.